In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import logging

for handler in logging.root.handlers:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

In [9]:
from customer_segmentation_toolkit.data_zoo import download_data_csv

from featurologists.features.customer_segmentation import transform
from featurologists.models.customer_segmentation import (
    calc_score_roc_auc,
    train_test_split,
    save_model,
    train_lightgbm,
    train_xgboost,
)

In [16]:
no_live_data = download_data_csv('data/output/01_data_split_offline_online/no_live_data.csv',
                                datetime_columns=["InvoiceDate"])
no_live_data.head()

INFO:root:Downloading dataset 'https://raw.githubusercontent.com/artemlops/customer-segmentation-toolkit/master/data/output/01_data_split_offline_online/no_live_data.csv'


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
df = transform(no_live_data)
df.head()

,CustomerID,count,min,max,mean,sum,categ_0,categ_1,categ_2,categ_3,categ_4,LastPurchase,FirstPurchase,cluster
0,12347.0,5,382.52,711.79,558.172000,2790.86,32.408290,10.442659,33.948317,14.524555,8.676179,59,297,4
1,12348.0,4,227.44,892.80,449.310000,1797.24,0.000000,38.016069,61.983931,0.000000,0.000000,5,288,1
2,12350.0,1,334.40,334.40,334.400000,334.40,0.000000,11.692584,60.406699,27.900718,0.000000,240,240,1
3,12352.0,6,144.35,840.30,345.663333,2073.98,16.427352,0.491808,66.125517,3.370331,13.584991,2,226,1
4,12353.0,1,89.00,89.00,89.000000,89.00,0.000000,0.000000,13.033708,19.887640,67.078652,134,134,0


In [28]:
from pathlib import Path
DATA = Path("../data/customer_segmentation")
DATA.mkdir(exist_ok=True)

df.to_csv(DATA / "no_live_data__transformed.csv")

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(df)

INFO:root:Received data of shape: (3611, 14), columns ['CustomerID', 'count', 'min', 'max', 'mean', 'sum', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4', 'LastPurchase', 'FirstPurchase', 'cluster']


In [20]:
xgboost_model = train_xgboost(X_train, Y_train)
xgboost_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='aucpr',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.16, max_delta_step=0,
              max_depth=50, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [21]:
xgboost_score = calc_score_roc_auc(xgboost_model, X_test, Y_test)
print(f"XGBoost score_roc_auc: {xgboost_score:.6f}")

XGBoost score_roc_auc: 0.925846


In [22]:
lightgbm_model = train_lightgbm(X_train, Y_train)
lightgbm_model

/home/ay/.pyenv/versions/3.7.6/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [23]:
lightgbm_score = calc_score_roc_auc(lightgbm_model, X_test, Y_test)
print(f"LightGBM score_roc_auc: {lightgbm_score:.6f}")

LightGBM score_roc_auc: 0.921138


In [26]:
from pathlib import Path

! pwd
PROJECT_ROOT = Path("..").resolve()
PROJECT_ROOT_LS = [p.name for p in PROJECT_ROOT.iterdir()]
assert "featurologists" in PROJECT_ROOT_LS, f"Not a project root? {PROJECT_ROOT}"

MODELS_DIR = PROJECT_ROOT / "models" / "customer_segmentation"
! rm -r {MODELS_DIR}

save_model(
    xgboost_model, MODELS_DIR / "xgboost", metadata={"score_roc_auc": xgboost_score}
)
save_model(
    lightgbm_model, MODELS_DIR / "lightgbm", metadata={"score_roc_auc": lightgbm_score}
)

[p.name for p in MODELS_DIR.iterdir()]

/plain/github/opensource/Featurologists/notebooks


['lightgbm', 'xgboost']